# Instructions for these exercises
> ## You are welcome to use any of the methods/scripts discussed in the demos. These exercises can be implemented in more than one way. Use whichever one works for you!
> ## Attempt as many of the following as you can. Do not worry if the last exercise, for example, doesn't work.
>
> ## Please add comments/discussion to your code to explain what you've done. Spanish is fine!
>
> ## When you are done, save this notebook (File --> Download --> Download .ipynb) and attach the document as your submission to the Moodle platform.
> ### Instructions on how to do this: [in English](https://www.youtube.com/watch?v=pRWA9XoTIJQ) and [en Español](https://www.youtube.com/watch?v=Xi5fJYEtCV8).
---


## ADQL resources:
>
1.   The [ADQL cookbook](https://www.gaia.ac.uk/data/gaia-data-release-1/adql-cookbook) hosted by the UK Gaia collaboration
2.   The NAVO workshop (Github repository with Python notebooks [here](https://github.com/nasa-navo/navo-workshop), video [here](https://www.youtube.com/watch?v=fIvTQ2iZscU))
3.   SIMBAD's [ADQL cheatsheet] (http://simbad.u-strasbg.fr/simbad/tap/help/adqlHelp.html). You can also check out some of their examples [here] (http://simbad.u-strasbg.fr/simbad/sim-tap) (click on the dropdown menu under "ADQL QUERY TO EXECUTE (or choose an example)").
4.   Vizier's [ADQL cheatsheet] (http://tapvizier.u-strasbg.fr/adql/help.html)

In [ ]:
# Some basics are installed/imported for you
try:
  import pyvo as vo
except ImportError:
  %pip install pyvo
  import pyvo as vo

import numpy as np

r = vo.regsearch(servicetype = 'tap', keywords = ['Simbad'])
SIMBAD_service = r[0].service

r = vo.regsearch(servicetype = 'tap', keywords = ['Vizier'])
# There are two servers, the backup server doesn't allow uploads. Choose the correct one.
k = int(np.nonzero(['obstap' not in rr['ivoid'] for rr in r])[0]) # avoid the column corresponding to the url that doesn't accept uploads
Vizier_service = r[k].service

from io import BytesIO
from astropy.io import votable

     |████████████████████████████████| 802 kB 4.9 MB/s 
  Created wheel for mimeparse: filename=mimeparse-0.1.3-py3-none-any.whl size=3864 sha256=080738b56ad21b8bc7d0d330ff9235b28e87e3e8f32a88f4f8e3b0a0dfd0e2e7
  Stored in directory: /root/.cache/pip/wheels/49/b4/2d/0081759ae1833bd694024801f7aacddcda8a687e8d5fbaeebd
Successfully built mimeparse


# 1. Find the SIMBAD entry for the globular cluster "omega Centauri"
> ### Columns to retrieve: `RA`, `DEC`, `main_id`, and `oid`
> ### Print out the main identifer (`main_id`). Note that it is not "omega Centauri".

In [ ]:
# Fill in the appropriate query here
query = """
SELECT basic.OID,
       RA,
       DEC,
       main_id
FROM basic JOIN ident ON oidref = oid
WHERE id = 'omega Centauri'
"""

result1 = SIMBAD_service.search(query).to_table()
print("The SIMBAD main identifier for the cluster is {}".format(result1['main_id'][0]))
result

The SIMBAD main identifier for the cluster is NGC  5139


oid,ra,dec,main_id
,deg,deg,
int64,float64,float64,object
3375707,201.697,-47.47947222222223,NGC 5139


# 2a. Do a bibliography search and find the most recent paper that studied the Cluster
> ### Specifically, find the latest paper that has the phrases "tidal radius" and "%omega%Cen" in its `abstract`.
> ### The query should extract all the information necessary to construct a URL to the paper's entry on the ADS (see example in Demo Session 2).

In [ ]:
# Fill in the appropriate query here
query = """
SELECT TOP 1 bibcode
FROM ref
WHERE abstract LIKE '%omega%Cen%' AND abstract LIKE '%tidal radius%'
ORDER BY "year" DESC
"""

result2 = SIMBAD_service.search(query).to_table()
print("The url is https://ui.adsabs.harvard.edu/abs/{}".format(result2['bibcode'][0]))

The url is https://ui.adsabs.harvard.edu/abs/2017AJ....153..175C


# 2b. Finding the tidal radius (approximate angular size) of the cluster
> ### Follow the URL to the PDF of the most recent paper. Consult Table 1 of the paper. Here, you will find the value of the tidal radius of the cluster in arcmin.
> ### Change the value of the variable `radius_arcmin` to the appropriate value below
> ### If you couldn't find the tidal radius, let `` be set to 60 arcmin as below.

In [ ]:
#Note that the step of finding the tidal radius is MANUAL! This is the only manual step in the exercises.
radius_arcmin = 57.03 # Change this to the radius you find it in Table 1 of the paper
radius_deg = str(radius_arcmin / 60.)

# 3. Perform a cone search for C-, M-, or K-type stars within `radius_deg` of the cluster centre.
> ### Columns to retrieve: `oid`, `main_id`, and `sp_type` (spectral type of all the stars within the matching radius).
> ### Hint: check that stars are within the radius using `CONTAINS`, AND restrict their spectral types with `(sp_type LIKE 'K%' or sp_type LIKE 'C%' or sp_type LIKE 'M%')`
>
> ### How many sources are in your final table?

In [ ]:
# Fill in the appropriate query here
query = """
SELECT oid, main_id, sp_type
FROM basic
WHERE CONTAINS(POINT('ICRS', RA, DEC), CIRCLE('ICRS', 201.697, -47.47947222222223, """ + radius_deg + """)) = 1
AND sp_type LIKE 'K%' OR sp_type LIKE 'C%' OR sp_type LIKE 'M%'
"""

result3 = SIMBAD_service.search(query).to_table()
# write code below to tell me how many sources there are in result3
len(result3)

50000

# 4. Find the 2MASS counterparts, if available, for each source
> ### Check SIMBAD's `ident` table to see if the `oid` corresponding to each row in `result3` has a counterpart in the 2MASS All-Sky Point Source Catalogue.
> ### Hint: for each `oid`, find the row in `ident` such that `ident.id` starts WITH '2MASS J'
>
> ### How many sources from `result3` do NOT have 2MASS counterparts?

In [ ]:
# Fill in the appropriate query here
# Collect the 2MASS identifiers into a column named `TMASS_ID`
query = """

"""

# results3 is now uploaded to the SIMBAD TAP server as `mytable`, and it should
# be used in the query above.
votable_object = BytesIO()
votable.writeto(votable.from_table(results3), votable_object)
votable_object.seek(0)

result4 = SIMBAD_service.search(query, uploads = {'mytable': votable_object}).to_table()

# write code below to tell me how many sources from `results3` don't have 2MASS counterparts in `results4`.

# Extra line: modify the 2MASS identifier column in `results4` to remove the "2MASS J" prefix
#             This is done to correctly match the format in the 2MASS point-source catalogue table (see next cell)
result4['TMASS_ID'] = [rr['TMASS_ID'].replace('2MASS J', '') for rr in result4]

# 5. Obtain the 2MASS photometry from VizieR
> ### Select the columns `magJ` and `magK` from the 2MASS point-source catalogue on VizieR (table ID: `'II/246/out'`)
> ### Hint: for each `oid`, find the row in `'II/246/out'` such that `TMASS_ID` from `results4` is equal to `"2MASS"` in `'II/246/out'`

In [ ]:
Vizier_table_id = 'II/246/out'
# Fill in the appropriate query here
query = """

"""

# results3 is now uploaded to the SIMBAD TAP server as `mytable`, and it should
# be used in the query above.
votable_object = BytesIO()
votable.writeto(votable.from_table(results4), votable_object)
votable_object.seek(0)

result5 = Vizier_service.search(query, uploads = {'mytable': votable_object}).to_table()

# write code below to plot a K vs J-K colour-magnitude diagram showing all the sources in result5.


